In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth, dayofweek, avg, count, lit, stddev
from pyspark.sql.window import Window

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator


bronze_path = "Files/bronze/transacciones_servicios/"
silver_path = "Files/silver/transacciones_servicios/"
gold_path = "Files/gold/dataset_recurrencia_servicios/"

modelo_fecha_path = "Files/models/modelo_servicios_fecha/"
modelo_monto_path = "Files/models/modelo_servicios_monto/"

notebook_version = "1.0"

print(f"Ruta Bronze: {bronze_path}")
print(f"Ruta Silver: {silver_path}")
print(f"Ruta Gold: {gold_path}")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 76, Finished, Available, Finished)

Ruta Bronze: Files/bronze/transacciones_servicios/
Ruta Silver: Files/silver/transacciones_servicios/
Ruta Gold: Files/gold/dataset_recurrencia_servicios/


In [ ]:
df_modelo_servicios = spark.sql("SELECT * FROM fs_servicios")
                                   
print(f"Lectura completada. Se leyeron {df_modelo_servicios.count()} registros.")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 77, Finished, Available, Finished)

Lectura completada. Se leyeron 3563259 registros.


In [ ]:
MIN_TX_COUNT = 10
MAX_STD_DIAS = 2
MAX_ITER = 10
MAX_DEPTH = 10

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 78, Finished, Available, Finished)

In [ ]:
print("Creando perfiles de comportamiento por par...")

df_perfiles = df_modelo_servicios.groupBy("cedula_remitente", "numero_cuenta_servicio").agg(
    count(lit(1)).alias("total_transacciones"),
    stddev("dias_diferencia_anterior_transaccion").alias("std_dias_diferencia")
)

df_pares_predecibles = df_perfiles.filter(
    (col("total_transacciones") >= MIN_TX_COUNT) &
    (col("std_dias_diferencia") <= MAX_STD_DIAS)
)

print(f"Total de pares únicos en los datos: {df_perfiles.count()}")
print(f"Número de pares 'predecibles' identificados: {df_pares_predecibles.count()}")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 79, Finished, Available, Finished)

Creando perfiles de comportamiento por par...
Total de pares únicos en los datos: 117194
Número de pares 'predecibles' identificados: 47


In [ ]:
print("Creando el dataset de entrenamiento 'élite'...")

# Hacemos un 'join' para quedarnos solo con las transacciones de los pares predecibles
# Un 'semi join' es eficiente para este tipo de filtrado
df_entrenamiento_elite = df_modelo_servicios.join(
    df_pares_predecibles, 
    on=["cedula_remitente", "numero_cuenta_servicio"], 
    how="semi"
)

print(f"Tamaño del dataset original: {df_modelo_servicios.count()}")
print(f"Tamaño del dataset élite para entrenamiento: {df_entrenamiento_elite.count()}")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 80, Finished, Available, Finished)

Creando el dataset de entrenamiento 'élite'...
Tamaño del dataset original: 3563259
Tamaño del dataset élite para entrenamiento: 1267


In [ ]:
df_features_fecha_data = df_entrenamiento_elite.select('dias_diferencia_anterior_transaccion',
                                                        'monto',
                                                        'dia_transaccion',
                                                        'dia_semana',
                                                        'mes_transaccion',
                                                        'conteo_transacciones_hist',
                                                        'promedio_dias_hist',
                                                        'label_fecha')

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 81, Finished, Available, Finished)

In [ ]:
features_fecha = [
    'dias_diferencia_anterior_transaccion',
    'monto',
    'dia_transaccion',
    'dia_semana',
    'mes_transaccion',
    'conteo_transacciones_hist',
    'promedio_dias_hist'
]

assembler_fecha = VectorAssembler(inputCols=features_fecha, outputCol="features", handleInvalid="skip")

# Vectorizacion de las features
df_fecha_model_data = assembler_fecha.transform(df_features_fecha_data)

(train_data_fecha, test_data_fecha) = df_fecha_model_data.randomSplit([0.8, 0.2], seed=42)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 82, Finished, Available, Finished)

In [ ]:
print("registros de entrenamiento -> ", train_data_fecha.count())
print("registros de testeo -> ", test_data_fecha.count())

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 83, Finished, Available, Finished)

registros de entrenamiento ->  1000
registros de testeo ->  267


In [ ]:
gbt_fecha = GBTRegressor(featuresCol="features", 
                         labelCol="label_fecha", 
                         maxIter=MAX_ITER)

model_fecha = gbt_fecha.fit(train_data_fecha)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 84, Finished, Available, Finished)

In [ ]:
print("iniciando predicciones de testing ...")

predictions_fecha = model_fecha.transform(test_data_fecha)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 85, Finished, Available, Finished)

iniciando predicciones de testing ...


In [ ]:
display(predictions_fecha.select("label_fecha", "prediction"))

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 86, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7b651ffa-a763-481a-b0e1-bb93ccf6cc66)

In [ ]:
evaluator_rmse_fecha = RegressionEvaluator(
    labelCol="label_fecha",
    predictionCol="prediction",
    metricName="rmse"
)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 87, Finished, Available, Finished)

In [ ]:
rmse_fecha = evaluator_rmse_fecha.evaluate(predictions_fecha)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 88, Finished, Available, Finished)

In [ ]:
print("Error Cuadrado Medio de las predicciones es -> ", rmse_fecha, "dias")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 89, Finished, Available, Finished)

Error Cuadrado Medio de las predicciones es ->  34.19703215182613 dias


In [ ]:
model_fecha.write().overwrite().save(modelo_fecha_path)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 90, Finished, Available, Finished)

# Modelo Predictivo para el Monto

In [ ]:
features_monto = [
    'monto',
    'monto_anterior_transaccion',
    'dias_diferencia_anterior_transaccion',
    'dia_transaccion',
    'conteo_transacciones_hist',
    'promedio_monto_hist'
]

assembler_monto = VectorAssembler(inputCols=features_monto, outputCol="features", handleInvalid="skip")

df_monto_model_data = assembler_monto.transform(df_entrenamiento_elite)

print("Dataset con la columna 'features' ensamblada:")
df_monto_model_data.select("features", "label_monto").show(5, truncate=False)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 91, Finished, Available, Finished)

Dataset con la columna 'features' ensamblada:
+---------------------------------------------------+-----------+
|features                                           |label_monto|
+---------------------------------------------------+-----------+
|[1840.88,1457.64,33.0,16.0,21.0,1355.3576190476188]|1762.49    |
|[1762.49,1840.88,28.0,14.0,22.0,1373.8636363636363]|1988.95    |
|[1084.18,1053.33,34.0,24.0,12.0,1072.7458333333334]|1053.33    |
|[1078.01,1736.36,31.0,16.0,15.0,1370.7973333333332]|1213.75    |
|[1053.33,1053.33,32.0,24.0,5.0,1067.374]           |1084.18    |
+---------------------------------------------------+-----------+
only showing top 5 rows



In [ ]:
(train_data_monto, test_data_monto) = df_monto_model_data.randomSplit([0.7, 0.3], seed=456)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 92, Finished, Available, Finished)

In [ ]:
print("Entrenando modelo de MONTO...")
gbt_monto = GBTRegressor(featuresCol="features", labelCol="label_monto", maxIter=10)
model_monto = gbt_monto.fit(train_data_monto)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 93, Finished, Available, Finished)

Entrenando modelo de MONTO...


In [ ]:
print("iniciando predicciones de testing ...")

predictions_monto = model_monto.transform(test_data_monto)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 94, Finished, Available, Finished)

iniciando predicciones de testing ...


In [ ]:
display(predictions_monto.select("label_monto", "prediction"))

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 95, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 856817dd-73b0-4933-a5bd-0696a54caecd)

In [ ]:
evaluator_rmse_monto = RegressionEvaluator(
    labelCol="label_monto", 
    predictionCol="prediction", 
    metricName="rmse")


StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 96, Finished, Available, Finished)

In [ ]:
rmse_monto = evaluator_rmse_monto.evaluate(predictions_monto)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 97, Finished, Available, Finished)

In [ ]:
print("Error Cuadrado Medio de las predicciones es -> ", rmse_monto, "monto")

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 98, Finished, Available, Finished)

Error Cuadrado Medio de las predicciones es ->  7575.94978956858 monto


In [ ]:
model_monto.write().overwrite().save(modelo_monto_path)

StatementMeta(, d3fdc1f7-346a-46f8-8125-2327b908e293, 99, Finished, Available, Finished)